In [1]:
import pandas as pd
import numpy as np

In [5]:
# Load the dataset
file_path = 'C:\\Users\\rloke\\OneDrive\\Desktop\\Datasets\\LosAngles_Crimes.csv'  # Replace with your file path
la_crimes = pd.read_csv(file_path)


In [6]:
print(la_crimes.isnull().sum())

DR_NO                  0
Date Rptd              0
DATE OCC               0
TIME OCC               0
AREA                   0
AREA NAME              0
Rpt Dist No            0
Part 1-2               0
Crm Cd                 0
Crm Cd Desc            0
Mocodes           114148
Vict Age               0
Vict Sex          108529
Vict Descent      108537
Premis Cd             10
Premis Desc          488
Weapon Used Cd    537498
Weapon Desc       537498
Status                 0
Status Desc            0
Crm Cd 1              10
Crm Cd 2          764505
Crm Cd 3          823173
Crm Cd 4          825151
LOCATION               0
Cross Street      693343
LAT                    0
LON                    0
dtype: int64


In [8]:
# Convert date columns to datetime format with explicit format
date_format = '%Y-%m-%d'  # Replace this with the actual format of your date strings
la_crimes['Date Rptd'] = pd.to_datetime(la_crimes['Date Rptd'], format=date_format)
la_crimes['DATE OCC'] = pd.to_datetime(la_crimes['DATE OCC'], format=date_format)

# Rest of your code...

# Split 'TIME OCC' into 'Hour OCC' and 'Minute OCC'
la_crimes['Hour OCC'] = la_crimes['TIME OCC'] // 100
la_crimes['Minute OCC'] = la_crimes['TIME OCC'] % 100


In [10]:
 #Merge 'Crm Cd' columns
crm_cd_columns = ['Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4']
la_crimes['Crm Cds'] = la_crimes[crm_cd_columns].apply(lambda row: [cd for cd in row if not np.isnan(cd)], axis=1)
la_crimes = la_crimes.drop(columns=crm_cd_columns)

# Drop columns with high missing values
columns_to_drop = ['Mocodes', 'Cross Street']
la_crimes = la_crimes.drop(columns=columns_to_drop)

In [14]:
# Columns identified for removal
essential_columns_to_drop = ['TIME OCC','AREA','Rpt Dist No','Part 1-2']


# Drop the specified columns
la_crimes = la_crimes.drop(columns=essential_columns_to_drop)


In [11]:
# Remove outliers based on IQR (similar to box plot method)
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

numeric_outlier_columns = ['LAT', 'LON','Vict Age','Hour OCC','Minute OCC']
for col in numeric_outlier_columns:
    la_crimes = remove_outliers(la_crimes, col)

In [16]:
# Specify the file path for the new CSV
output_file_path = 'LosAngles_crimes_cleaned.csv'  # You can change this to your desired file path

# Save the cleaned DataFrame to a CSV file
la_crimes.to_csv(output_file_path, index=False)
